In [1]:
pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=95cc9d133555402ad7d9e18cb94b2d88b8a95172b9cfcf5b374b12b097dbed83
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn

IMG_SHAPE = 224
batch_size = 32
data_dir = '/kaggle/input/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/'

transform = transforms.Compose([
    transforms.Resize((IMG_SHAPE, IMG_SHAPE)),
    transforms.ToTensor(),
])

all_ds = ImageFolder(root=data_dir, transform=transform)

print(all_ds)

total_size = len(all_ds)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_ds, rest_ds = random_split(all_ds, [train_size, total_size - train_size])
val_ds, test_ds = random_split(rest_ds, [val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=4)


Dataset ImageFolder
    Number of datapoints: 4200
    Root location: /kaggle/input/tuberculosis-tb-chest-xray-dataset/TB_Chest_Radiography_Database/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [25]:
class weighted_binary_crossentropy(nn.Module):
    def __init__(self, class_ratio):
        super(weighted_binary_crossentropy, self).__init__()
        self.class_ratio = class_ratio

    def forward(self, y_true, y_pred, epsilon=1e-7):
        weight_0 = torch.tensor([1 / self.class_ratio]).to(y_true.device)
        weight_1 = torch.tensor([1.0]).to(y_true.device)

        loss_pos = weight_0 * y_true * torch.log(y_pred + epsilon)
        loss_neg = weight_1 * (1 - y_true) * torch.log(1 - y_pred + epsilon)

        return -torch.mean(loss_pos + loss_neg)



In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [45]:
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self, img_width, img_height):
        super(MyModel, self).__init__()
        input_shape = (3, img_width, img_height)

        self.model = torchvision.models.efficientnet_b7(weights = 'EfficientNet_B7_Weights.IMAGENET1K_V1')

        for param in self.model.parameters():
            param.requires_grad = False

        dense_out_features = torchvision.models.efficientnet_b7(weights = 'EfficientNet_B7_Weights.IMAGENET1K_V1').classifier[1].out_features  # Get the output features of the dense net
        self.layer256 = nn.Linear(dense_out_features, 32)
        self.relu = nn.ReLU()
        self.pReLU = nn.PReLU()
        self.m = nn.BatchNorm1d(256)
        self.layer32 = nn.Linear(512, 256)
        self.layer8 = nn.Linear(32, 8)
        self.layer1 = nn.Linear(32, 1)

        self.Sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = self.layer256(x)
        #x = self.pReLU(x)
        #x = self.layer32(x)
        #x = self.pReLU(x)
        #x = self.layer8(x)
        #x = self.pReLU(x)
        x = self.layer1(x)
        x = self.Sigmoid(x)
        return x


In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from efficientnet_pytorch import EfficientNet

model = MyModel(500, 500).to(device)

Loss = weighted_binary_crossentropy(5)

optimizer_conv = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for i, (inputs, labels) in enumerate(train_loader):

        # Move the batch tensor to the device (e.g., GPU)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer_conv.zero_grad()

        outputs = model(inputs)

        loss = Loss(outputs, labels)

        loss.backward()

        optimizer_conv.step()

        epoch_loss += loss.item()

        predicted = (outputs >= 0.5).float()  # Threshold the outputs

        #correct_preds += (predicted == labels).sum().item()  # Accumulate correct predictions

        for i in range(labels.size(0)):
          if predicted[i] == labels[i] :
            correct_preds += 1

        total_preds += labels.size(0)  # Accumulate total predictions

    # Calculate accuracy for the epoch
    accuracy = correct_preds / total_preds
    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/20, Average Loss: 2.6625, Accuracy: 0.6262
Epoch 2/20, Average Loss: 2.6463, Accuracy: 0.6925
Epoch 3/20, Average Loss: 2.6394, Accuracy: 0.7330
Epoch 4/20, Average Loss: 2.6359, Accuracy: 0.7636
Epoch 5/20, Average Loss: 2.6286, Accuracy: 0.7735
Epoch 6/20, Average Loss: 2.6333, Accuracy: 0.7412
Epoch 7/20, Average Loss: 2.6194, Accuracy: 0.7463
Epoch 8/20, Average Loss: 2.6274, Accuracy: 0.7697
Epoch 9/20, Average Loss: 2.6091, Accuracy: 0.8024
Epoch 10/20, Average Loss: 2.6148, Accuracy: 0.7779
Epoch 11/20, Average Loss: 2.6162, Accuracy: 0.7786
Epoch 12/20, Average Loss: 2.6066, Accuracy: 0.7816
Epoch 13/20, Average Loss: 2.6124, Accuracy: 0.7884
Epoch 14/20, Average Loss: 2.6273, Accuracy: 0.7908
Epoch 15/20, Average Loss: 2.6230, Accuracy: 0.8007
Epoch 16/20, Average Loss: 2.6300, Accuracy: 0.8126
Epoch 17/20, Average Loss: 2.6195, Accuracy: 0.8051
Epoch 18/20, Average Loss: 2.6153, Accuracy: 0.8099
Epoch 19/20, Average Loss: 2.6134, Accuracy: 0.8112
Epoch 20/20, Average 

In [47]:
def predict():
    correct_preds = 0
    total_preds = 0
    tens_label = torch.tensor([]).to(device)  # Move tens tensor to the same device as inputs and labels
    tens_input = torch.tensor([]).to(device)  # Move tens tensor to the same device as inputs and labels

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            tens_label = torch.cat((labels, tens_label), 0)
            
            outputs = model(inputs)

            predicted = (outputs >= 0.5).float()  
            
            tens_input = torch.cat((predicted, tens_input), 0)
            
            for i in range(labels.size(0)):
                if predicted[i] == labels[i] :
                    correct_preds += 1

            total_preds += labels.size(0) 

    accuracy = correct_preds / total_preds
        
    return accuracy ,tens_input ,tens_label


In [48]:
ac ,inp ,lb = predict()

In [49]:
inp = inp.cpu().numpy()
lb = lb.cpu().numpy()

In [43]:
from sklearn.metrics import accuracy_score

total_right = accuracy_score(lb, inp, normalize=False)
print(total_right)
accuracy = (total_right / len(lb))
print(accuracy)

480
0.7619047619047619


In [50]:
from sklearn.metrics import classification_report

print(classification_report(lb, inp))

              precision    recall  f1-score   support

         0.0       0.89      0.90      0.89       522
         1.0       0.49      0.46      0.47       108

    accuracy                           0.82       630
   macro avg       0.69      0.68      0.68       630
weighted avg       0.82      0.82      0.82       630



In [51]:
from sklearn.metrics import confusion_matrix

confusion_matrix(lb, inp)

array([[469,  53],
       [ 58,  50]])